In [ ]:
class CNN_QNetwork(tf.keras.Model):
    def __init__(self, action_size):
        super(CNN_QNetwork, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, 4, activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(action_size)

    def call(self, x):
        x = tf.cast(x, tf.float32)
        x = tf.expand_dims(x, -1)  # shape (batch, 6, 7, 1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        return self.output_layer(x)


class PERMemory:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.priorities = []
        self.alpha = 0.6

    def add(self, experience, error):
        max_priority = max(self.priorities, default=1.0)
        self.buffer.append(experience)
        self.priorities.append(max_priority)
        if len(self.buffer) > self.capacity:
            self.buffer.pop(0)
            self.priorities.pop(0)

    def sample(self, batch_size):
        scaled_priorities = np.array(self.priorities) ** self.alpha
        probs = scaled_priorities / np.sum(scaled_priorities)
        indices = np.random.choice(len(self.buffer), batch_size, p=probs)
        samples = [self.buffer[i] for i in indices]
        return samples, indices

    def update(self, indices, errors):
        for i, e in zip(indices, errors):
            self.priorities[i] = abs(e) + 1e-6